### Загружаем базовую модель

In [ ]:
from vlm_finetune import AutoVlmModel, ImageProcessor


model = AutoVlmModel.from_name(
    model_name="llava", 
    model_path="llava-hf/llava-1.5-7b-hf",
    image_processor=ImageProcessor() # можно заресайзить картинки, чтобы сэкономить память
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Тестим на нашем датасете, для примера возьмём рандомный сэмпл

In [2]:
import json

with open("data/vet_dataset_test.json", "r") as file:
    sample = json.loads(file.read())[0]

In [3]:
sample

{'image_path': 'data/грудная/52а.png',
 'answer': 'легкие расправлены без очаговых и инфильтрированных теней. легочной рисунок не изменен. тень сердца и аорты без изменений. просвет трахеи не нарушен. уплотнение хрящей гортани, увеличение в объеме.  на момент проведения исследования выявлены признаки ларингита'}

In [ ]:
prompt = "Поставь вероятный диагноз по рентгенограмме."

In [5]:
%%time

model_answer = model.predict(image=sample["image_path"], prompt=prompt)

CPU times: user 1.57 s, sys: 879 ms, total: 2.44 s
Wall time: 2.58 s


In [6]:
model_answer

'Поставь вероятный диагноз по рентгенограмме.'

In [7]:
import evaluate

bertscore = evaluate.load("bertscore")

In [8]:
bertscore.compute(predictions=[model_answer], references=[sample["answer"]], lang="ru")

{'precision': [0.6366536021232605],
 'recall': [0.5959105491638184],
 'f1': [0.6156086325645447],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.57.1)'}

**Модель ответила абсолютно бессвязно, просто повторила наш вопрос**

### Зафайнтюним базовую модель с помощью фреймворка
Параметры берём дефолтные (они уже зашиты во фреймворк), параметры можно менять и "докручивать"

In [9]:
model.finetune(
    dataset_path="data/vet_dataset_train.json", 
    output_dir="models/llava_tuned", 
    prompt=prompt, 
    num_train_epochs=3
)

2025-10-29 17:36:53 [INFO] vlm_finetune.base.model: Используется устройство: cuda:0
2025-10-29 17:36:53 [INFO] vlm_finetune.base.model: Количество обучаемых параметров: 19136512
2025-10-29 17:36:53 [INFO] vlm_finetune.base.model: Начинаем finetune...


trainable params: 19,136,512 || all params: 7,082,563,584 || trainable%: 0.2702


Step,Training Loss
10,2.240100
20,1.402400


2025-10-29 18:53:08 [INFO] vlm_finetune.base.model: Модель успешно затюнена и сохранена в models/llava_tuned
2025-10-29 18:53:08 [INFO] vlm_finetune.base.model: !Для предсказаний будет использоваться доученная модель!


### Смотрим тот же сэмпл из нашего датасета

In [10]:
%%time

model_answer = model.predict(image=sample["image_path"], prompt=prompt)

CPU times: user 14.4 s, sys: 7.33 s, total: 21.7 s
Wall time: 21.4 s


In [11]:
model_answer

'Тени органов грудной полости визуализируются хорошо. Просвет трахеи не нарушен. Легкие расширены без очаговых и инфильтратированных теней, бронхиальный тип нагрузки. Тень сердца и аорты без изменений. Просвет пищевода не нарушен. На момент исследования выявлены признаки бронхита.'

In [12]:
bertscore.compute(predictions=[model_answer], references=[sample["answer"]], lang="ru")

{'precision': [0.8219882249832153],
 'recall': [0.809727132320404],
 'f1': [0.8158116340637207],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.57.1)'}


**Модель отвечает по теме**